In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial import ConvexHull
from sklearn.neighbors import NearestNeighbors
scaler=StandardScaler()

In [63]:
df = pd.read_csv('../RL data/1004nuance_CCRES_mu-_p_gamma_gamma.csv',index_col=0)
df=df.reset_index(drop=True)
df.head()

,Unnamed: 0.1,isTrueTrack,nuance,pdg,completeness,purity,energy,momentum,vertex,x,z,hitEnergies,hitWidths,event
0,77935,0,1004,22,0.927203,1.000000,0.142519,0.0580184 0.109872 -0.0698122,89.9957 32.1891 362.521,128.152 125.666 120.141 120.271 119.483 120.39...,312.25 314.65 324.85 325.15 325.45 325.45 325....,41.5782 35.1936 60.8753 89.6193 81.5882 84.586...,0.389073 0.391386 0.4822 0.575952 0.647041 0.4...,186
1,77936,0,1004,22,0.962025,1.000000,0.075676,-0.038573 0.0563472 0.032618,89.9957 32.1891 362.521,73.2294 73.1715 72.5502 73.4597 72.3008 73.491...,377.05 377.35 377.65 377.65 377.95 377.95 378....,187.472 391.158 276.34 322.829 336.921 243.674...,0.427461 0.642389 0.58376 0.436167 0.570565 0....,186
2,77937,1,1004,13,1.000000,1.000000,0.544252,-0.475466 -0.0248804 0.241578,89.9957 32.1891 362.521,89.6898 89.1138 88.5655 87.9245 87.4135 86.876...,362.95 363.25 363.55 363.85 364.15 364.45 364....,151.157 149.134 152.551 174.052 206.565 161.39...,0.858241 0.761823 0.826274 0.795586 0.659256 0...,186
3,77938,1,1004,2212,1.000000,1.000000,1.442270,0.59796 -0.0723102 0.914877,89.9957 32.1891 362.521,90.5144 90.4481 90.8394 91.0402 91.2357 91.417...,362.35 362.65 362.95 363.25 363.55 363.85 364....,48.1207 204.197 109.501 104.111 115.941 111.57...,0.484322 0.769033 0.470719 0.476296 0.476078 0...,186
4,78186,0,1004,22,0.727768,0.966265,0.277808,0.108591 -0.160076 0.199402,78.8851 20.1652 196.674,89.4671 89.5251 89.6635 89.8183 89.9743 90.122...,215.05 215.35 215.65 215.95 216.25 216.55 216....,71.8066 184.974 208.823 219.392 228.083 210.89...,0.430399 0.434834 0.446315 0.449988 0.452191 0...,296


In [64]:
x=[]
z=[]
x_scaled=[]
z_scaled=[]
v_x=[]
v_y=[]
v_z=[]
m_x=[]
m_y=[]
m_z=[]

for j in range(len(df)):
    print(j)
    x.append([float(i) for i in df['x'][j][0:-1].split(' ')])
    z.append([float(i) for i in df['z'][j][0:-1].split(' ')])
    x_scaled.append(scaler.fit_transform(np.array(x[j]).reshape(-1,1)).reshape(1,-1)[0])
    z_scaled.append(scaler.fit_transform(np.array(z[j]).reshape(-1,1)).reshape(1,-1)[0])
    vertex= [float(i) for i in df['vertex'][j].split(' ')]
    momentum=[float(i) for i in df['momentum'][j].split(' ')]
    v_x.append(vertex[0])
    v_y.append(vertex[1])
    v_z.append(vertex[2])
    m_x.append(momentum[0])
    m_y.append(momentum[1])
    m_z.append(momentum[2])

df['x']=x
df['z']=z
df['x_scaled']=x_scaled
df['z_scaled']=z_scaled
df['v_x']=v_x
df['v_y']=v_y
df['v_z']=v_z
df['m_x']=m_x
df['m_y']=m_y
df['m_z']=m_z


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [65]:
#df=df[(df.completeness>0.8) & df.purity>0.7]
df.head()
#len(df)

,Unnamed: 0.1,isTrueTrack,nuance,pdg,completeness,purity,energy,momentum,vertex,x,...,hitWidths,event,x_scaled,z_scaled,v_x,v_y,v_z,m_x,m_y,m_z
0,77935,0,1004,22,0.927203,1.000000,0.142519,0.0580184 0.109872 -0.0698122,89.9957 32.1891 362.521,"[128.152, 125.666, 120.141, 120.271, 119.483, ...",...,0.389073 0.391386 0.4822 0.575952 0.647041 0.4...,186,"[3.36946139716, 2.69930741444, 1.20992657997, ...","[-3.60185185185, -3.1450617284, -1.2037037037,...",89.9957,32.1891,362.521,0.058018,0.109872,-0.069812
1,77936,0,1004,22,0.962025,1.000000,0.075676,-0.038573 0.0563472 0.032618,89.9957 32.1891 362.521,"[73.2294, 73.1715, 72.5502, 73.4597, 72.3008, ...",...,0.427461 0.642389 0.58376 0.436167 0.570565 0....,186,"[1.31444785791, 1.29942614027, 1.1382345484, 1...","[-1.29591472169, -1.21991370794, -1.1439126941...",89.9957,32.1891,362.521,-0.038573,0.056347,0.032618
2,77937,1,1004,13,1.000000,1.000000,0.544252,-0.475466 -0.0248804 0.241578,89.9957 32.1891 362.521,"[89.6898, 89.1138, 88.5655, 87.9245, 87.4135, ...",...,0.858241 0.761823 0.826274 0.795586 0.659256 0...,186,"[1.62858014463, 1.60608901616, 1.5846794923, 1...","[-1.62881911248, -1.60670636126, -1.5845936100...",89.9957,32.1891,362.521,-0.475466,-0.024880,0.241578
3,77938,1,1004,2212,1.000000,1.000000,1.442270,0.59796 -0.0723102 0.914877,89.9957 32.1891 362.521,"[90.5144, 90.4481, 90.8394, 91.0402, 91.2357, ...",...,0.484322 0.769033 0.470719 0.476296 0.476078 0...,186,"[-1.42085814318, -1.42562538156, -1.3974893275...","[-1.56985101134, -1.55324781574, -1.5366446201...",89.9957,32.1891,362.521,0.597960,-0.072310,0.914877
4,78186,0,1004,22,0.727768,0.966265,0.277808,0.108591 -0.160076 0.199402,78.8851 20.1652 196.674,"[89.4671, 89.5251, 89.6635, 89.8183, 89.9743, ...",...,0.430399 0.434834 0.446315 0.449988 0.452191 0...,296,"[-1.04088372768, -1.0359690087, -1.02424147237...","[-1.20411172782, -1.18632633642, -1.1685409450...",78.8851,20.1652,196.674,0.108591,-0.160076,0.199402


In [66]:
def RSS(df):
    rssList = []
    x = [np.array(df['x_scaled'][i]) for i in range(len(df))]
    z = [np.array(df['z_scaled'][i]) for i in range(len(df))]
    for i in range(len(x)):
        
        perp_dist=0
        length=len(z[i])
        reg = LinearRegression().fit(x[i].reshape(-1,1), z[i])
        y_pred = reg.predict(x[i].reshape(-1,1))
        for j in range(length):
            perp_dist+=abs(-reg.coef_[0]*x[i][j] + z[i][j] -reg.intercept_)/(reg.coef_[0]**2 + 1)**0.5
        rssList.append(perp_dist/length)
        print(i,perp_dist/length )
    return rssList

In [67]:
RSS=RSS(df)


0 0.0888040396935
1 0.0593058293706
2 0.00406474026071
3 0.0699139353046
4 0.162054518599
5 0.00906529149533
6 0.364775458489
7 0.0905793219488
8 0.443692357477
9 0.419988139968
10 0.107403799459
11 0.0152631454158
12 0.155334200926
13 0.522610840888
14 0.0767940055608
15 0.0403514638392
16 0.054891013839
17 0.438752089669
18 0.0959781920057
19 0.0824086968619
20 0.252910550072
21 0.270368871335
22 0.117355204419
23 0.0451619261015
24 0.326582299135
25 0.241526217758
26 0.0159024934778
27 0.34036405516
28 0.719991200738
29 0.274566982304
30 0.0185093516542
31 0.0429061622032
32 0.0759606295375
33 0.641394275422
34 0.0158747629531
35 0.0261063207862
36 0.195110748111
37 0.173842212716
38 0.112912996239
39 0.0354857476866
40 0.411133245703
41 0.263182282585
42 0.0190641774686
43 0.15787780853
44 0.101053917157
45 0.0091047135053
46 0.0450072804058
47 0.214148003509
48 0.111000521567
49 0.320237818859
50 0.00769215244047
51 0.0408935745706
52 0.363851649323
53 0.0636045723018
54 0.0214769

In [68]:
def convexHullArea(df):
    areaList=[]
    x = [np.array(df['x_scaled'][i]) for i in range(len(df))]
    z = [np.array(df['z_scaled'][i]) for i in range(len(df))]
    for i in range(len(x)):
        data = [[x[i][j], z[i][j]] for j in range(len(x[i]))]
        #print(data,i)
        #print(i, area)
        try:
            hull = ConvexHull(data)
            areaList.append(hull.area)
        except:
            areaList.append(0)
        print(i, hull.area)
    return areaList

In [69]:
area=convexHullArea(df)


0 14.96228932814975
1 11.350989312089279
2 9.859496614057942
3 9.896736173839587
4 12.693284540793993
5 9.557192830471973
6 11.09004136505516
7 8.4101339820554
8 16.672572602286237
9 9.312636778179433
10 9.774386795377774
11 9.38179127347785
12 10.656802989358827
13 15.15114143019729
14 9.669489890991969
15 9.37344517305054
16 9.78954855719486
17 11.126331894135692
18 10.009706180190998
19 9.514357298972959
20 12.753573155744682
21 10.190256264088646
22 9.21361855747998
23 9.154612726576127
24 10.648008973872338
25 13.529746832547762
26 9.667039812650552
27 10.668296700625511
28 9.595886151657675
29 17.647877178141663
30 9.572271322091705
31 10.046649847734438
32 7.130195762868186
33 13.643561485420708
34 9.678536140092413
35 9.798234263186448
36 9.568889732700228
37 8.882430808538526
38 11.135224854753972
39 8.774262292949677
40 23.55566063953499
41 12.86880832227279
42 10.177295696653703
43 9.22453324988968
44 9.243368667008845
45 10.22127355795304
46 9.553972535490336
47 15.85921827

In [45]:
def length(df):
    lengthList = []
    scaler = StandardScaler()
    df2 = df.copy(deep=True)
    x = [np.array(df['x'][i]) for i in df.index]
    z = [np.array(df['z'][i]) for i in df.index]
    for i in df.index:
        print(i)
        #x[i] = scaler.fit_transform(np.array(ast.literal_eval(df2['x'][i])).reshape(-1,1))
        #z[i] = scaler.fit_transform(np.array(ast.literal_eval(df2['z'][i])).reshape(-1,1))
        x[i] = np.array(ast.literal_eval(df2['x'][i]))
        z[i] = np.array(ast.literal_eval(df2['z'][i]))
        data = [[x[i][j], z[i][j]] for j in range(len(x[i]))]
        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[1])
        pca = PCA(n_components=1)
        new_data = pca.fit_transform(data)
        length = float(max(new_data)- min(new_data))
        lengthList.append(length)
        print(length)
    return lengthList

In [70]:
def findEpsBinSearch(df):
    epsList=[]
    x = [np.array(df['x_scaled'][i]) for i in range(len(df))]
    z = [np.array(df['z_scaled'][i]) for i in range(len(df))]
    for i in range(len(x)):
        #print(i)
        data = [[x[i][j], z[i][j]] for j in range(len(x[i]))]
        eps = 0.001
        epsRange=np.arange(0,20, 0.001)
        l = 0
        h= len(epsRange)-1
        
        while l<=h:
            m= int((l+h)/2)
            clustering = DBSCAN(eps=epsRange[m], min_samples=2).fit(data)
            if (list(clustering.labels_).count(0)/len(clustering.labels_)< 1): 
                l = m + 1
            elif (list(clustering.labels_).count(0)/len(clustering.labels_) == 1): 
                h = m - 1
        
        epsList.append(epsRange[int((l+h)/2)])
        print(i,epsRange[int((l+h)/2)] )
    return epsList

In [71]:
epsList = findEpsBinSearch(df)



0 2.446
1 2.317
2 0.522
3 0.558
4 1.566
5 0.158
6 1.095
7 0.642
8 3.254
9 0.55
10 0.792
11 0.318
12 1.385
13 1.586
14 0.205
15 1.147
16 1.091
17 0.457
18 0.084
19 0.313
20 2.673
21 0.373
22 0.255
23 0.25
24 0.437
25 2.536
26 0.126
27 0.655
28 0.59
29 4.005
30 0.183
31 0.059
32 2.282
33 1.949
34 0.034
35 0.148
36 0.548
37 0.497
38 0.159
39 0.398
40 6.243
41 2.262
42 0.27
43 0.081
44 1.818
45 0.627
46 0.327
47 2.798
48 0.346
49 0.796
50 0.102
51 0.904
52 1.851
53 0.395
54 0.259
55 0.257
56 1.106
57 2.025
58 0.042
59 0.34
60 1.625
61 1.204
62 0.1
63 0.744
64 1.565
65 0.753
66 0.198
67 0.312
68 1.892
69 0.342
70 0.048
71 0.663
72 1.561
73 1.276
74 0.048
75 0.373
76 1.074
77 1.699
78 0.017
79 0.196
80 1.163
81 1.842
82 0.367
83 0.477
84 4.37
85 2.774
86 0.44
87 0.211
88 1.52
89 1.115
90 0.058
91 0.098
92 0.326
93 0.632
94 0.442
95 0.087
96 1.255
97 0.809
98 0.089
99 1.73
100 2.051
101 2.392
102 0.218
103 0.284
104 2.95
105 0.247
106 0.491
107 0.184
108 0.37
109 1.975
110 0.071
111 0.318
112

In [70]:
def pc2Var(df):
    pc2VarList = []
    scaler = StandardScaler()
    df2 = df.copy(deep=True)
    x = [np.array(df['x'][i]) for i in range(len(df))]
    z = [np.array(df['z'][i]) for i in range(len(df))]
    for i in range(len(df)):
        print(i)
        x[i] = scaler.fit_transform(x[i].reshape(-1,1))
        z[i] = scaler.fit_transform(z[i].reshape(-1,1))
        data = [[x[i][j], z[i][j]] for j in range(len(x[i]))]
        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[1])
        pca = PCA(n_components=1)
        pca.fit_transform(data)
        pc2VarList.append(1- pca.explained_variance_ratio_)
    return pc2VarList

In [72]:
def slidingWindowHelper(x,y, n):
    
    reg=LinearRegression().fit(x.reshape(-1,1), y)
    reg.coef_, reg.intercept_
    d=0.3
    m = reg.coef_[0]
    
    
    if m == 0:
        return 0

    dx= d/(m*m +1)**0.5 
    
    def y_pred(x):
        return float(m*x+reg.intercept_)
    
    intercepts = [(1/float(m))*i + y_pred(i) for i in np.arange(min(x),max(x),float(dx)) ]
    #print(len(intercepts), m)
    perp_dist=0
    #plt.scatter(x, y)
    #plt.show()

    for i in range(0,len(intercepts)-1):
        bighits=[]
        smallhits=[]
        
        if m > 0:
            for j in range(len(x)):
                if y[j]<-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]<-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
        else:
            for j in range(len(x)):
                if y[j]>-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]>-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
            
        #print(smallhits)
        if len(smallhits)==0:
            continue
        new_x = np.array([x[k] for k in bighits])
        new_y = [y[k] for k in bighits]
        
        new_reg=LinearRegression().fit(new_x.reshape(-1,1), new_y)
        for l in smallhits:
            perp_dist+=abs(-new_reg.coef_[0]*x[l] + y[l] -new_reg.intercept_)/(new_reg.coef_[0]**2 + 1)**0.5
    return perp_dist/len(x)      

In [73]:
def slidingWindow(df,n):
    slidingWindowList=[]
    x = [np.array(df['x'][i]) for i in range(len(df))]
    z = [np.array(df['z'][i]) for i in range(len(df))]
    for i in range(len(x)):
        
        #print(df.x[i], df.z[i])
        answer = slidingWindowHelper(x[i],z[i], n)
        print(i,answer)
        slidingWindowList.append(answer)
    return slidingWindowList

In [74]:
sliding = slidingWindow(df,4)



0 0.19340000806
1 0.130036956525
2 0.0143026866787
3 0.0182955971887
4 0.222885267997
5 0.0113180763525
6 0.336079976399
7 0.0204822647859
8 0.0888345320188
9 0.0985532795082
10 0.087360453488
11 0.0176414348593
12 0.248622976566
13 0.38101557158
14 0.0391979060899
15 0.0152960215954
16 0.137786949366
17 0.209896691171
18 0.0142078434288
19 0.00451543093154
20 0.305879141592
21 0.0430881399699
22 0.0106162419962
23 0.00559613888754
24 0.293600281543
25 0.109711488456
26 0.00769055195703
27 0.0121762738991
28 0.105493192984
29 0.340359890544
30 0.00961300369036
31 0.0148583278181
32 0.0994215321775
33 0.425929657269
34 0.00569657135958
35 0.00939720260576
36 0.130311036939
37 0.106964627696
38 0.0113167016599
39 0.00686958561205
40 0.485718953914
41 0.0254215446125
42 0.00944391389231
43 0.125759280568
44 0.111474617484
45 0.0154047108609
46 0.0339005456917
47 0.0111846684827
48 0.116833956273
49 0.292340763009
50 0.0189599094216
51 0.00582438111916
52 0.203955791459
53 0.066788497851
5

In [59]:
nHits = [len(df['x'][i]) for i in range(len(df)) ]

In [60]:
df['eps'] =epsList


df['sliding']=sliding




df['nHits']=nHits

df['RSS']=RSS


df['envelope']=area




In [61]:
df.head()

,Unnamed: 0.1,isTrueTrack,nuance,pdg,completeness,purity,energy,momentum,vertex,x,...,v_y,v_z,m_x,m_y,m_z,eps,sliding,nHits,RSS,envelope
0,77588,1,1003,2212,0.995614,1.000000,1.170380,-0.417132 0.283818 0.484651,213.372 94.5945 628.466,"[213.169, 212.868, 212.607, 212.331, 212.09, 2...",...,94.59450,628.466,-0.417132,0.283818,0.484651,0.079,0.012381,76,0.026129,9.792788
1,77589,1,1003,13,1.000000,0.995902,0.243355,-0.0621655 -0.192301 -0.0849337,213.372 94.5945 628.466,"[195.941, 196.005, 196.175, 196.5, 196.908, 19...",...,94.59450,628.466,-0.062165,-0.192301,-0.084934,0.097,0.015039,64,0.076023,9.404007
2,77590,1,1003,211,1.000000,1.000000,0.356428,0.295004 -0.0404061 0.137483,213.372 94.5945 628.466,"[215.357, 215.794, 216.59, 217.201, 217.165, 2...",...,94.59450,628.466,0.295004,-0.040406,0.137483,0.152,0.178248,56,0.074120,9.675034
3,77598,1,1003,13,0.986159,1.000000,0.417528,0.22993 -0.157998 0.292121,219.522 7.86647 141.993,"[220.939, 221.169, 221.419, 221.714, 221.87, 2...",...,7.86647,141.993,0.229930,-0.157998,0.292121,0.038,0.016541,167,0.013988,9.877187
4,77599,1,1003,2212,0.968750,1.000000,1.058730,0.000676978 0.0646625 0.486186,219.522 7.86647 141.993,"[220.135, 220.171, 220.184, 220.058, 220.053, ...",...,7.86647,141.993,0.000677,0.064662,0.486186,0.717,0.010108,36,0.184861,10.822871


In [62]:
df.to_csv('../RL data/vars1004nuance_CCRES_mu-_p_gamma_gamma.csv')